# Download

In [42]:
WORKING_ENV = 'LOCAL' # Can be LABS, COLAB or PAPERSPACE
assert WORKING_ENV in ['LABS', 'COLAB', 'LOCAL']

import sys
if WORKING_ENV == 'COLAB':
  from google.colab import drive
  drive.mount('/content/drive/')
  !pip install medmnist
  !pip install torch
  !pip install gputil
  !pip install psutil
  !pip install humanize
  ROOT = "/content/drive/MyDrive/ColabNotebooks/med-contrastive-project/"
  sys.path.append(ROOT + "./swav/")
  !nvidia-smi
elif WORKING_ENV == 'LABS':
  ROOT = "/vol/bitbucket/sx119/Contrastive-Medical-Image-Classification/"
else:
  ROOT = "/Users/xushitong/Contrastive-Medical-Image-Classification/"

# Import

In [43]:
import medmnist

import argparse
import math
import os
import shutil
import time
from logging import getLogger

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torchvision
import torchvision.transforms as transforms
import tqdm
# import loader

from utils import (
    bool_flag,
    initialize_exp,
    restart_from_checkpoint,
    fix_random_seeds,
    AverageMeter,
    init_distributed_mode,
)
from multicropdataset import MultiCropDataset
import resnet50 as resnet_models



In [ ]:
# Import packages
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))

mem_report()

CPU RAM Free: 8.7 GB
GPU 0 ... Mem Free: 6329MB / 15360MB | Utilization  57%


# Hyperparameters

In [44]:
EPOCH_NUM = 20
BATCH_SIZE = 16
LEARNING_RATE = 4.8
TRAIN_SET_RATIO = 0.9
CROP_NUM = [2]
CROP_SIZE = [224]
MIN_SCALE_CROP = [0.14]
MAX_SCALE_CROP = [1]

trial_name = f"epochs{EPOCH_NUM}_batch{BATCH_SIZE}_lr{LEARNING_RATE}_crop-num{CROP_NUM}-size{CROP_SIZE}_scale-crop-min{MIN_SCALE_CROP}-max{MAX_SCALE_CROP}"
arg_command = \
f"--epochs {EPOCH_NUM} --batch_size {BATCH_SIZE} --base_lr {LEARNING_RATE}\
 --nmb_crops {' '.join(map(str, CROP_NUM))} --size_crops {' '.join(map(str, CROP_SIZE))} --min_scale_crops {' '.join(map(str, MIN_SCALE_CROP))} --max_scale_crops {' '.join(map(str, MAX_SCALE_CROP))}\
 --data_path {ROOT}./datasets".split(" ")

print(f"Running command {arg_command}")


Running command ['--epochs', '20', '--batch_size', '16', '--base_lr', '4.8', '--nmb_crops', '2', '--size_crops', '224', '--min_scale_crops', '0.14', '--max_scale_crops', '1', '--data_path', '/Users/xushitong/Contrastive-Medical-Image-Classification/./datasets']


In [45]:
# logger = getLogger()

parser = argparse.ArgumentParser(description="Implementation of SwAV")

#########################
#### data parameters ####
#########################
parser.add_argument("--data_path", type=str, default="/path/to/data",
                    help="path to dataset repository")
parser.add_argument("--nmb_crops", type=int, default=[2], nargs="+",
                    help="list of number of crops (example: [2, 6])")
parser.add_argument("--size_crops", type=int, default=[224], nargs="+",
                    help="crops resolutions (example: [224, 96])")
parser.add_argument("--min_scale_crops", type=float, default=[0.14], nargs="+",
                    help="argument in RandomResizedCrop (example: [0.14, 0.05])")
parser.add_argument("--max_scale_crops", type=float, default=[1], nargs="+",
                    help="argument in RandomResizedCrop (example: [1., 0.14])")

#########################
## swav specific params #
#########################
parser.add_argument("--crops_for_assign", type=int, nargs="+", default=[0, 1],
                    help="list of crops id used for computing assignments")
parser.add_argument("--temperature", default=0.1, type=float,
                    help="temperature parameter in training loss")
parser.add_argument("--epsilon", default=0.05, type=float,
                    help="regularization parameter for Sinkhorn-Knopp algorithm")
parser.add_argument("--sinkhorn_iterations", default=3, type=int,
                    help="number of iterations in Sinkhorn-Knopp algorithm")
parser.add_argument("--feat_dim", default=128, type=int,
                    help="feature dimension")
parser.add_argument("--nmb_prototypes", default=3000, type=int,
                    help="number of prototypes")
parser.add_argument("--queue_length", type=int, default=0,
                    help="length of the queue (0 for no queue)")
parser.add_argument("--epoch_queue_starts", type=int, default=15,
                    help="from this epoch, we start using a queue")

#########################
#### optim parameters ###
#########################
parser.add_argument("--epochs", default=100, type=int,
                    help="number of total epochs to run")
parser.add_argument("--batch_size", default=64, type=int,
                    help="batch size per gpu, i.e. how many unique instances per gpu")
parser.add_argument("--base_lr", default=4.8, type=float, help="base learning rate")
parser.add_argument("--final_lr", type=float, default=0, help="final learning rate")
parser.add_argument("--freeze_prototypes_niters", default=313, type=int,
                    help="freeze the prototypes during this many iterations from the start")
parser.add_argument("--wd", default=1e-6, type=float, help="weight decay")
parser.add_argument("--warmup_epochs", default=10, type=int, help="number of warmup epochs")
parser.add_argument("--start_warmup", default=0, type=float,
                    help="initial warmup learning rate")

#########################
#### dist parameters ###
#########################
parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up distributed
                    training; see https://pytorch.org/docs/stable/distributed.html""")
parser.add_argument("--world_size", default=-1, type=int, help="""
                    number of processes: it is set automatically and
                    should not be passed as argument""")
parser.add_argument("--rank", default=0, type=int, help="""rank of this process:
                    it is set automatically and should not be passed as argument""")
parser.add_argument("--local_rank", default=0, type=int,
                    help="this argument is not used and should be ignored")

#########################
#### other parameters ###
#########################
parser.add_argument("--arch", default="resnet50", type=str, help="convnet architecture")
parser.add_argument("--hidden_mlp", default=2048, type=int,
                    help="hidden layer dimension in projection head")
parser.add_argument("--workers", default=10, type=int,
                    help="number of data loading workers")
parser.add_argument("--checkpoint_freq", type=int, default=25,
                    help="Save the model periodically")
parser.add_argument("--use_fp16", type=bool_flag, default=True,
                    help="whether to train with mixed precision or not")
parser.add_argument("--sync_bn", type=str, default="pytorch", help="synchronize bn")
parser.add_argument("--syncbn_process_group_size", type=int, default=8, help=""" see
                    https://github.com/NVIDIA/apex/blob/master/apex/parallel/__init__.py#L58-L67""")
parser.add_argument("--dump_path", type=str, default=".",
                    help="experiment dump path for checkpoints and log")
parser.add_argument("--seed", type=int, default=31, help="seed")



_StoreAction(option_strings=['--seed'], dest='seed', nargs=None, const=None, default=31, type=<class 'int'>, choices=None, required=False, help='seed', metavar=None)

In [46]:
args = parser.parse_args(arg_command)
fix_random_seeds(args.seed)

# Dataset

In [47]:
train_dataset = MultiCropDataset(
        medmnist.PathMNIST("train", download=False, root=args.data_path),
        args.data_path,
        args.size_crops,
        args.nmb_crops,
        args.min_scale_crops,
        args.max_scale_crops,
    )

pretrain_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16)


In [ ]:
# train_dataset = medmnist.PathMNIST("train", download=False, root=args.data, 
#                                    transform=loader.TwoCropsTransform(transforms.Compose(augmentation)))
# val_dataset = medmnist.PathMNIST("val", download=False, root=args.data, 
#                                    transform=loader.TwoCropsTransform(transforms.Compose(augmentation)))

# pretrain_len = int(len(train_dataset) * TRAIN_SET_RATIO)
# pretrain_set, pretrain_val_set = torch.utils.data.random_split(train_dataset, [pretrain_len, len(train_dataset) - pretrain_len])

# pretrain_loader = torch.utils.data.DataLoader(
#     pretrain_set, batch_size=args.batch_size, shuffle=True, 
#     pin_memory=True, drop_last=True)


# Training Helper Functions

In [ ]:
if WORKING_ENV == 'LABS':
  summary = open(trial_name + ".txt", "a")
else:
  summary = sys.stdout

@torch.no_grad()
def distributed_sinkhorn(out):
    Q = torch.exp(out / args.epsilon).t() # Q is K-by-B for consistency with notations from our paper
    B = Q.shape[1] * args.world_size # number of samples to assign
    K = Q.shape[0] # how many prototypes

    # make the matrix sums to 1
    sum_Q = torch.sum(Q)
    # dist.all_reduce(sum_Q)
    Q /= sum_Q

    for it in range(args.sinkhorn_iterations):
        # normalize each row: total weight per prototype must be 1/K
        sum_of_rows = torch.sum(Q, dim=1, keepdim=True)
        # dist.all_reduce(sum_of_rows)
        Q /= sum_of_rows
        Q /= K

        # normalize each column: total weight per sample must be 1/B
        Q /= torch.sum(Q, dim=0, keepdim=True)
        Q /= B

    Q *= B # the colomns must sum to 1 so that Q is an assignment
    return Q.t()


def train(train_loader, model, optimizer, epoch, lr_schedule, queue):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    model.train()
    use_the_queue = False

    end = time.time()
    with tqdm.tqdm(pretrain_loader, unit="batch") as tepoch: 
        if WORKING_ENV == "LABS":
          tepoch = pretrain_loader
        for it, inputs in enumerate(tepoch):
            # measure data loading time
            data_time.update(time.time() - end)

            # update learning rate
            iteration = epoch * len(train_loader) + it
            for param_group in optimizer.param_groups:
                param_group["lr"] = lr_schedule[iteration]

            # normalize the prototypes
            with torch.no_grad():
                w = model.prototypes.weight.data.clone()
                w = nn.functional.normalize(w, dim=1, p=2)
                model.prototypes.weight.copy_(w)

            # ============ multi-res forward passes ... ============
            embedding, output = model(inputs)
            embedding = embedding.detach()
            bs = inputs[0].size(0)

            # ============ swav loss ... ============
            loss = 0
            for i, crop_id in enumerate(args.crops_for_assign):
                with torch.no_grad():
                    out = output[bs * crop_id: bs * (crop_id + 1)].detach()

                    # time to use the queue
                    if queue is not None:
                        if use_the_queue or not torch.all(queue[i, -1, :] == 0):
                            use_the_queue = True
                            out = torch.cat((torch.mm(
                                queue[i],
                                model.module.prototypes.weight.t()
                            ), out))
                        # fill the queue
                        queue[i, bs:] = queue[i, :-bs].clone()
                        queue[i, :bs] = embedding[crop_id * bs: (crop_id + 1) * bs]

                    # get assignments
                    q = distributed_sinkhorn(out)[-bs:]

                # cluster assignment prediction
                subloss = 0
                for v in np.delete(np.arange(np.sum(args.nmb_crops)), crop_id):
                    x = output[bs * v: bs * (v + 1)] / args.temperature
                    subloss -= torch.mean(torch.sum(q * F.log_softmax(x, dim=1), dim=1))
                loss += subloss / (np.sum(args.nmb_crops) - 1)
            loss /= len(args.crops_for_assign)

            # ============ backward and optim step ... ============
            optimizer.zero_grad()
            loss.backward()
            # cancel gradients for the prototypes
            if iteration < args.freeze_prototypes_niters:
                for name, p in model.named_parameters():
                    if "prototypes" in name:
                        p.grad = None
            optimizer.step()

            # ============ misc ... ============
            losses.update(loss.item(), inputs[0].size(0))
            batch_time.update(time.time() - end)
            end = time.time()
            if args.rank ==0 and it % 50 == 0:
                summary.write("Epoch: [{0}][{1}]\t"
                    "Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t"
                    "Data {data_time.val:.3f} ({data_time.avg:.3f})\t"
                    "Loss {loss.val:.4f} ({loss.avg:.4f})\t"
                    "Lr: {lr:.4f}".format(
                        epoch,
                        it,
                        batch_time=batch_time,
                        data_time=data_time,
                        loss=losses,
                        lr=optimizer.param_groups[0]["lr"],
                    )
                )
    return (epoch, losses.avg), queue



# Train

In [ ]:
# logger, training_stats = initialize_exp(args, "epoch", "loss")

# build data
# train_dataset = MultiCropDataset(
#     args.data_path,
#     args.size_crops,
#     args.nmb_crops,
#     args.min_scale_crops,
#     args.max_scale_crops,
# )
# train_loader = torch.utils.data.DataLoader(
#     train_dataset,
#     batch_size=args.batch_size,
#     num_workers=args.workers,
#     pin_memory=True,
#     drop_last=True
# )
# logger.info("Building data done with {} images loaded.".format(len(train_dataset)))

# build model
# model = torchvision.models.resnet50()
model = resnet_models.__dict__[args.arch](
        normalize=True,
        hidden_mlp=args.hidden_mlp,
        output_dim=args.feat_dim,
        nmb_prototypes=args.nmb_prototypes,
    )
# # synchronize batch norm layers
# model = nn.SyncBatchNorm.convert_sync_batchnorm(model)
# copy model to GPU
model = model.cuda()
# if args.rank == 0:
#     logger.info(model)
# logger.info("Building model done.")

# build optimizer
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=args.base_lr,
    momentum=0.9,
    weight_decay=args.wd,
)
# optimizer = LARC(optimizer=optimizer, trust_coefficient=0.001, clip=False)
warmup_lr_schedule = np.linspace(args.start_warmup, args.base_lr, len(pretrain_loader) * args.warmup_epochs)
iters = np.arange(len(pretrain_loader) * (args.epochs - args.warmup_epochs))
cosine_lr_schedule = np.array([args.final_lr + 0.5 * (args.base_lr - args.final_lr) * (1 + \
                      math.cos(math.pi * t / (len(pretrain_loader) * (args.epochs - args.warmup_epochs)))) for t in iters])
lr_schedule = np.concatenate((warmup_lr_schedule, cosine_lr_schedule))
# logger.info("Building optimizer done.")

# # wrap model
# model = nn.parallel.DistributedDataParallel(
#     model,
#     device_ids=[args.gpu_to_work_on]
# )

# optionally resume from a checkpoint
to_restore = {"epoch": 0}
# restart_from_checkpoint(
#     os.path.join(args.dump_path, "checkpoint.pth.tar"),
#     run_variables=to_restore,
#     state_dict=model,
#     optimizer=optimizer,
#     amp=apex.amp,
# )
start_epoch = to_restore["epoch"]

# build the queue
queue = None
queue_path = os.path.join(args.dump_path, "queue" + str(args.rank) + ".pth")
if os.path.isfile(queue_path):
    queue = torch.load(queue_path)["queue"]
# the queue needs to be divisible by the batch size
args.queue_length -= args.queue_length % (args.batch_size * args.world_size)

cudnn.benchmark = True

for epoch in range(start_epoch, args.epochs):

    # train the network for one epoch
    # logger.info("============ Starting epoch %i ... ============" % epoch)

    # # set sampler
    # train_loader.sampler.set_epoch(epoch)

    # optionally starts a queue
    if args.queue_length > 0 and epoch >= args.epoch_queue_starts and queue is None:
        queue = torch.zeros(
            len(args.crops_for_assign),
            args.queue_length // args.world_size,
            args.feat_dim,
        ).cuda()

    # train the network
    scores, queue = train(pretrain_loader, model, optimizer, epoch, lr_schedule, queue)
    # training_stats.update(scores)

    # # save checkpoints
    # if args.rank == 0:
    #     save_dict = {
    #         "epoch": epoch + 1,
    #         "state_dict": model.state_dict(),
    #         "optimizer": optimizer.state_dict(),
    #     }
    #     torch.save(
    #         save_dict,
    #         os.path.join(args.dump_path, "checkpoint.pth.tar"),
    #     )
    #     if epoch % args.checkpoint_freq == 0 or epoch == args.epochs - 1:
    #         shutil.copyfile(
    #             os.path.join(args.dump_path, "checkpoint.pth.tar"),
    #             os.path.join(args.dump_checkpoints, "ckp-" + str(epoch) + ".pth"),
    #         )
    # if queue is not None:
    #     torch.save({"queue": queue}, queue_path)





  0%|          | 1/5625 [00:00<1:00:20,  1.55batch/s]

Epoch: [0][0]	Time 0.646 (0.646)	Data 0.255 (0.255)	Loss 8.3943 (8.3943)	Lr: 0.0000

  1%|          | 33/5625 [00:21<59:50,  1.56batch/s]  


KeyboardInterrupt: ignored

In [ ]:
torch.save(model, f"{trial_name}.pickle")
mem_report()